AMUSE tutorial on high-order bridge
====================

Hierarchical coupling strategies are fundamental parts of AMUSE.
It enables us to combined the output of a wide variety of solvers into a homogeneous solution.
In this example we will be nesting multiple bridges, to show the power of bridge.

In [1]:
import numpy
from amuse.units import (units, constants)


In [2]:
def orbital_period(Mtot, a):
    return (((4 * numpy.pi**2) * a**3)/(constants.G * Mtot)).sqrt()

from amuse.ext.orbital_elements import new_binary_from_orbital_elements
def get_star_planet_and_moon():
    Mstar = 1.0|units.MSun
    Mplanet = 1.0|units.MJupiter
    a_planet = 5.2 | units.au
    e_planet = 0.0
    bodies = new_binary_from_orbital_elements(Mstar, Mplanet, a_planet, e_planet,
                                             G=constants.G)
    bodies[0].name = "star"
    bodies[1].name = "planet"
    planet = bodies[bodies.name=="planet"]
    RH_planet = a_planet*(1.0-e_planet)*(Mplanet/(3*Mstar))**(1./3.)
    a_moon = 0.1*RH_planet
    e_moon = 0.0
    Mmoon = 0.01*Mplanet
    Pmoon = orbital_period(Mplanet+Mmoon, a_moon)
    moon = new_binary_from_orbital_elements(planet.mass, 
                                            Mmoon, a_moon, e_moon,
                                            G=constants.G)
    moon.position -= moon[0].position
    moon.velocity -= moon[0].velocity
    moon = moon[1].as_set()
    moon.position += planet.position
    moon.velocity += planet.velocity
    moon.semimajor_axis = a_moon
    moon.name = "moon"
    bodies.add_particle(moon)
    bodies.move_to_center()
    return bodies
bodies = get_star_planet_and_moon()
print(bodies)

                 key         mass         name  semimajor_axis           vx           vy           vz            x            y            z
                   -         MSun         none  149597870691.0 * m  3646245880.3572216 * m * s**-1  3646245880.3572216 * m * s**-1  3646245880.3572216 * m * s**-1  149597870691.0 * m  149597870691.0 * m  149597870691.0 * m
====================  ===========  ===========  ===========  ===========  ===========  ===========  ===========  ===========  ===========
14080404950246810079    1.000e+00         star    0.000e+00    0.000e+00   -3.465e-09   -0.000e+00   -5.009e-03   -0.000e+00   -0.000e+00
 3778966842704923142    9.546e-04       planet    0.000e+00    0.000e+00    3.581e-06    0.000e+00    5.195e+00    0.000e+00    0.000e+00
 9412831179345245876    9.546e-06         moon    3.550e-02    0.000e+00    4.927e-06    0.000e+00    5.230e+00    0.000e+00    0.000e+00
====================  ===========  ===========  ===========  ===========  ==========

Now we have the orbits of the three particles, star, planet and moon. We now want to make a disk around the moon.

In [5]:
from amuse.ext.protodisk import ProtoPlanetaryDisk

from amuse.lab import nbody_system

def make_disk_around_celestial_body(moon, Mplanet):

    R = 1|units.au
    a_moon = moon.semimajor_axis
    e_moon = 0.0
    Mmoon = moon.mass.sum()
    RH_moon = a_moon*(1.0-e_moon)*(Mmoon/(3*Mplanet))**(1./3.)
    converter = nbody_system.nbody_to_si(Mmoon, R)
    Ndisk = 1000
    Rin = 0.03*RH_moon
    Rout = 0.3*RH_moon
    Pinner = orbital_period(Mmoon, Rin)
    Mdisk = 0.01 * Mmoon

    disk = ProtoPlanetaryDisk(Ndisk,
                              convert_nbody=converter,
                              Rmin=Rin/R,
                              Rmax=Rout/R,
                              q_out=10.0,
                              discfraction=Mdisk/Mmoon).result
    disk.move_to_center()
    disk.position += moon.position
    disk.velocity += moon.velocity

    masses = Mdisk/float(Ndisk)
    disk.mass = masses
    rho = 3.0| (units.g/units.cm**3)
    disk.radius = (disk.mass/(4*rho))**(1./3.)
    return disk, Pinner, converter

planet = bodies[bodies.name=="planet"]
moon = bodies[bodies.name=="moon"]
disk, Pinner, converter = make_disk_around_celestial_body(moon, 
                                               planet.mass.sum())

In [9]:
from amuse.community.huayno.interface import Huayno
gravityA = Huayno(converter)
gravityA.particles.add_particles(bodies)
channel = {"from stars": bodies.new_channel_to(gravityA.particles),
            "to_stars": gravityA.particles.new_channel_to(bodies)}

gravityB = Huayno(converter, mode="openmp")
gravityB.particles.add_particles(disk)
channel.update({"from_disk": disk.new_channel_to(gravityB.particles)})
channel.update({"to_disk": gravityB.particles.new_channel_to(disk)})
bodies.add_particles(disk)

In [10]:
from amuse.couple import bridge
from amuse.ext.composition_methods import *
gravhydro = bridge.Bridge(use_threading=False, method=SPLIT_4TH_S_M4)
gravhydro.add_system(gravityA, (gravityB,))
gravhydro.add_system(gravityB, (gravityA,))
gravhydro.timestep = 0.5*Pinner

In [11]:
from amuse.ext.composition_methods import *
from amuse.ext.orbital_elements import orbital_elements_from_binary

def gravity_hydro_bridge(gravityA, gravityB, gravhydro, bodies,
                         t_end):

    gravity_initial_total_energy = gravityA.get_total_energy() + gravityB.get_total_energy()
    model_time = 0 | units.Myr
    dt = 0.012|units.yr  #1.0*Pinner
    while model_time < t_end:

        model_time += dt

        orbit_planet = orbital_elements_from_binary(bodies[:2], G=constants.G)
        orbit_moon = orbital_elements_from_binary(bodies[1:3], G=constants.G)
        print("Planet:", "ae=", orbit_planet[2].in_(units.AU), orbit_planet[3])
        print("Moon:", "ae=", orbit_moon[2].in_(units.AU), orbit_moon[3])
        
        dE_gravity = gravity_initial_total_energy/(gravityA.get_total_energy()+gravityB.get_total_energy())
        print("Time:", model_time.in_(units.day), \
              "dE=", dE_gravity)#, dE_hydro

        gravhydro.evolve_model(model_time)
        channel["to_stars"].copy()
        channel["to_disk"].copy()
        print("S=", bodies[:3])
        print("g=", gravityA.particles)
        print(gravityA.particles.y.in_(units.au), stars.y.in_(units.au))
        
    gravityA.stop()
    gravityB.stop()

t_end = 1.0 | units.yr
gravity_hydro_bridge(gravityA, gravityB, gravhydro, 
                     bodies, t_end)

Planet: ae= 5.20234212361 AU 0.0013660333751
Moon: ae= 0.035504872805 AU 7.71982602908e-05
Time: 4.382906388 day dE= 1.0


CodeException: Exception when calling function 'evolve_model', of code 'HuaynoInterface', exception was 'lost connection to code'

You have created a 

Assignmnets and questions:
---------------

### Assignment 1:


### Question 1:
